In [1]:
# 2 parameters for this script
d = '../data/pre/' # data directory
format_train = True # True for train, False for test

# output
ffm_train_path = '%sffm_train' % d
ffm_test_path = '%sffm_test' % d

import os
import pandas as pd
import numpy as np
from collections import Counter
from time import time

new_train = '%snew_generated_train.csv' % d
new_test = '%snew_generated_test.csv' % d

In [2]:
tr = pd.read_csv(new_train, index_col=0)
te = pd.read_csv(new_test, index_col=0)
te['label'] = -1

In [4]:
if format_train:
    data = tr
else: 
    data = te

In [5]:
cat_cols = ['gender', 'education', 'marriageStatus', 'haveBaby',
        'appPlatform', 'sitesetID', 'positionType', 'connectionType', 'telecomsOperator']
all_cat_to_one_hot = []

for c in cat_cols:
    all_cat_to_one_hot.append(pd.get_dummies(data[c], prefix=c))

concat = pd.concat(all_cat_to_one_hot, axis=1)

In [6]:
ffm_raw = pd.concat([data[['label', 'age']]]+all_cat_to_one_hot+
                   [data[['hometown', 'residence', 'adID', 'camgaignID', 'advertiserID', 'appID','clickTime',
        'conversionTime']]], axis=1)
del concat

In [7]:
from collections import OrderedDict
fields = tuple(list(OrderedDict.fromkeys(i.split('_')[0] for i in ffm_raw.columns))) # including 'label'
features = ffm_raw.columns

In [ ]:
def ffm_format(row):
    str_ = '1' if int(row[0])==1 else '-1'
    for i,v in enumerate(row[1:]):
        str_ += '\t%d:%d:%s' % (fields.index(features[i+1].split('_')[0])-1, i, str(v))
    return str_
    
formatted = ffm_raw.apply(ffm_format, axis=1)
formatted.to_csv(ffm_train_path if format_train else ffm_test_path, header=False, index=False)
        